In [1]:
import os
import copy
import numpy as np
from astropy import constants as const
from astropy.coordinates import SkyCoord, Angle, EarthLocation, Longitude, spherical_to_cartesian, cartesian_to_spherical
from astropy.time import Time, TimeDelta
from astropy import units
from pyuvdata import UVData
import pyuvdata.utils as uvutils
from pyuvdata.data import DATA_PATH

In [2]:
file1 = os.path.join(DATA_PATH, '1133866760.uvfits')

In [3]:
uvd1 = UVData()
uvd1.read_uvfits(file1)

In [4]:
phase_frame = 'gcrs'

In [5]:
print(uvd1.uvw_array.dtype)
uvd1.uvw_array = np.float64(uvd1.uvw_array)

uvd1_drift = copy.deepcopy(uvd1)
uvd1_unphase_phase = copy.deepcopy(uvd1)
uvd1_drift.unphase_to_drift(phase_frame=phase_frame, use_mwatools_phasing=False)
uvd1_unphase_phase.unphase_to_drift(phase_frame=phase_frame, use_mwatools_phasing=False)
uvd1_unphase_phase.phase(uvd1.phase_center_ra, uvd1.phase_center_dec, phase_frame=phase_frame, use_mwatools_phasing=False)

print(uvd1.uvw_array.dtype)
print(uvd1_unphase_phase.uvw_array.dtype)

print(uvd1.uvw_array - uvd1_unphase_phase.uvw_array)
print(np.max(np.abs(uvd1.uvw_array - uvd1_unphase_phase.uvw_array)))

temp = uvd1.uvw_array - uvd1_unphase_phase.uvw_array
wh_max = np.where(np.abs(temp) == np.max(np.abs(temp)))
print('drift baseline number with max difference:', wh_max)
print('drift baseline vector with max difference:', uvd1_drift.uvw_array[wh_max[0], :])
print('baseline length for max difference:', np.linalg.norm(uvd1_drift.uvw_array[wh_max[0], :]))

float32
float64
float64
[[-5.28594232e-11  7.25843234e-06 -2.35157958e-06]
 [-9.02097442e-06  1.71384086e-06  9.79367980e-06]
 [-2.05862264e-05 -7.49280231e-06 -4.24603688e-06]
 ...
 [-5.28594232e-11  7.25843234e-06 -2.35157958e-06]
 [ 1.91783239e-06  7.39906726e-06  8.65367441e-06]
 [-5.28594232e-11  7.25843234e-06 -2.35157958e-06]]
7.488040364478366e-05
drift baseline number with max difference: (array([3766]), array([0]))
drift baseline vector with max difference: [[-238.12780988  475.33967484   -3.67797623]]
baseline length for max difference: 531.6636040063181


In [6]:
print(np.min(uvd1.time_array) - np.max(uvd1.time_array))

0.0


In [7]:
obs_time = Time(uvd1.time_array[0], format='jd')

In [8]:
itrs_telescope_location = SkyCoord(x=uvd1.telescope_location[0] * units.m,
                                   y=uvd1.telescope_location[1] * units.m,
                                   z=uvd1.telescope_location[2] * units.m,
                                   representation='cartesian',
                                   frame='itrs', obstime=obs_time)
itrs_lat_lon_alt = uvd1.telescope_location_lat_lon_alt

frame_telescope_location = itrs_telescope_location.transform_to(phase_frame)

In [9]:
print(uvd1.uvw_array.dtype)
print(uvd1.uvw_array)

float64
[[ -0.          -0.          -0.        ]
 [ 48.95267868  10.95962811  21.54682732]
 [ 55.97493362   7.73503256  23.69575119]
 ...
 [ -0.          -0.          -0.        ]
 [ 75.37077332 -78.14148712  13.9833231 ]
 [ -0.          -0.          -0.        ]]


In [10]:
uvw_rel_positions = uvutils.unphase_uvw(uvd1.phase_center_ra, uvd1.phase_center_dec, uvd1.uvw_array)
uvw_rel_positions = np.float64(uvw_rel_positions)

rephase_uvws = uvutils.phase_uvw(uvd1.phase_center_ra, uvd1.phase_center_dec, uvw_rel_positions)


In [11]:
print(uvw_rel_positions.dtype)
print(rephase_uvws.dtype)

print(rephase_uvws - uvd1.uvw_array)
print(np.max(np.abs(rephase_uvws - uvd1.uvw_array)))

float64
float64
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-7.10542736e-15  0.00000000e+00 -3.55271368e-15]
 [-7.10542736e-15  8.88178420e-16  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.42108547e-14  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
4.547473508864641e-13


In [12]:
zenith_coord = SkyCoord(alt=Angle(90 * units.deg), az=Angle(0 * units.deg),
                        obstime=obs_time, frame='altaz',
                        location=EarthLocation.from_geocentric(uvd1.telescope_location[0],
                                                               uvd1.telescope_location[1],
                                                               uvd1.telescope_location[2],
                                                               unit='m'))
frame_zenith = zenith_coord.transform_to(phase_frame)
frame_ha = Longitude(frame_zenith.ra - Angle(uvd1.phase_center_ra * units.rad))

uvw_rot_positions = uvutils.mwatools_calcuvw_unphase(frame_ha.rad, uvd1.phase_center_dec, uvd1.uvw_array)
uvw_rot_positions = np.float64(uvw_rot_positions)

rephase_uvws_mwatools = uvutils.mwatools_calcuvw(frame_ha.rad, uvd1.phase_center_dec, uvw_rot_positions)

print(rephase_uvws_mwatools - uvd1.uvw_array)
print(np.max(np.abs(rephase_uvws_mwatools - uvd1.uvw_array)))

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.10542736e-15  0.00000000e+00  3.55271368e-15]
 [ 0.00000000e+00 -8.88178420e-16  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.42108547e-14 -3.55271368e-15]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
4.547473508864641e-13


In [13]:
print(uvw_rel_positions)

[[ 0.00000000e+00 -0.00000000e+00  0.00000000e+00]
 [ 1.18800866e+00  5.44518216e+01  3.78572680e+00]
 [-7.97878225e-01  6.12688569e+01  5.58155516e-02]
 ...
 [ 0.00000000e+00 -0.00000000e+00  0.00000000e+00]
 [-4.13440268e+01  6.39345156e+01 -7.86474918e+01]
 [ 0.00000000e+00 -0.00000000e+00  0.00000000e+00]]


In [14]:
frame_uvw_coord = SkyCoord(x=uvw_rel_positions[:, 0] * units.m + frame_telescope_location.cartesian.x,
                           y=uvw_rel_positions[:, 1] * units.m + frame_telescope_location.cartesian.y,
                           z=uvw_rel_positions[:, 2] * units.m + frame_telescope_location.cartesian.z,
                           representation='cartesian',
                           frame=phase_frame, obstime=obs_time)


In [15]:
print(frame_uvw_coord.cartesian.get_xyz().T[1:5, :])

[[ 5697076.41965917   108205.34099726 -2857822.28254363]
 [ 5697074.43377229   108212.15803255 -2857826.01245487]
 [ 5697070.81188996   108221.90596489 -2857832.77875173]
 [ 5697066.51328497   108213.58207792 -2857841.70894478]] m


In [16]:
itrs_uvw_coord = frame_uvw_coord.transform_to('itrs')
rt_frame_uvw_coord = itrs_uvw_coord.transform_to(phase_frame)

In [17]:
print(rt_frame_uvw_coord.cartesian - frame_uvw_coord.cartesian)
print(np.max(np.abs(rt_frame_uvw_coord.cartesian.get_xyz().value - frame_uvw_coord.cartesian.get_xyz().value)))

[( 0.00000000e+00,  5.82076609e-11, -7.62986019e-06),
 (-7.03707337e-06, -1.74633169e-05,  5.58234751e-06),
 (-4.23286110e-06,  4.22501762e-06,  1.36187300e-05), ...,
 ( 0.00000000e+00,  5.82076609e-11, -7.62986019e-06),
 (-1.02445483e-05, -1.64515368e-05,  2.81725079e-06),
 ( 0.00000000e+00,  5.82076609e-11, -7.62986019e-06)] m
8.10356141300872e-05


In [18]:
print(itrs_uvw_coord.cartesian.get_xyz().T[1:5, :])

[[-2559670.52320215  5095758.32615428 -2849012.14807288]
 [-2559675.81651662  5095753.59631885 -2849015.88139019]
 [-2559683.04871526  5095746.12949083 -2849022.6537806 ]
 [-2559673.69021568  5095745.85975562 -2849031.59019075]] m


In [19]:
drift_uvw_array = uvutils.ENU_from_ECEF(itrs_uvw_coord.cartesian.get_xyz().value,
                                                *itrs_lat_lon_alt).T


In [20]:
print(drift_uvw_array.dtype)
print(drift_uvw_array)

float64
[[ 2.11709077e-05 -6.25697504e-06  8.87108922e-06]
 [ 5.44198605e+01  4.37568570e+00  2.76596914e-01]
 [ 6.12730082e+01  2.08917292e-01  3.00821283e-01]
 ...
 [ 2.11709077e-05 -6.25697504e-06  8.87108922e-06]
 [ 6.46998171e+01 -8.82959831e+01 -3.68686208e-01]
 [ 2.11709077e-05 -6.25697504e-06  8.87108922e-06]]


In [21]:
print(uvd1_drift.uvw_array.dtype)
print(drift_uvw_array - uvd1_drift.uvw_array)
print(np.max(np.abs(drift_uvw_array - uvd1_drift.uvw_array)))

float64
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.82957734e-04  1.83811049e-03  2.67551243e-03]
 [-3.91896335e-05  2.06956415e-03  3.08315335e-03]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.99964647e-03  2.17332190e-03  4.74498322e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
0.11607322491789773


In [22]:
uvw_ecef = uvutils.ECEF_from_ENU(drift_uvw_array.T, *itrs_lat_lon_alt).T

In [23]:
print(uvw_ecef)
print(uvw_ecef.dtype)
print('')
print(uvw_ecef - itrs_uvw_coord.cartesian.get_xyz().value.T)
print(np.max(np.abs(uvw_ecef - itrs_uvw_coord.cartesian.get_xyz().value.T)))

[[-2559620.90003977  5095780.77518919 -2849015.93286817]
 [-2559670.52320215  5095758.32615428 -2849012.14807288]
 [-2559675.81651662  5095753.59631885 -2849015.88139019]
 ...
 [-2559620.90003977  5095780.77518919 -2849015.93286817]
 [-2559660.75978417  5095715.98636404 -2849094.64746636]
 [-2559620.90003977  5095780.77518919 -2849015.93286817]]
float64

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
0.0


In [24]:
new_itrs_uvw_coord = SkyCoord(x=uvw_ecef[:, 0] * units.m,
                          y=uvw_ecef[:, 1] * units.m,
                          z=uvw_ecef[:, 2] * units.m,
                          representation='cartesian',
                          frame='itrs', obstime=obs_time)

In [25]:
print(new_itrs_uvw_coord.cartesian.get_xyz() - itrs_uvw_coord.cartesian.get_xyz())
print(np.max(np.abs(new_itrs_uvw_coord.cartesian.get_xyz().value - itrs_uvw_coord.cartesian.get_xyz().value)))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] m
0.0


In [26]:
new_frame_uvw_coord = new_itrs_uvw_coord.transform_to(phase_frame)

In [27]:
new_frame_uvw_coord.representation = 'cartesian'
print(new_frame_uvw_coord)

<SkyCoord (GCRS: obstime=2457367.9577083588, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
    [(5697075.23165051, 108150.88917568, -2857826.06827805),
     (5697076.41965213, 108205.34097979, -2857822.28253804),
     (5697074.43376805, 108212.15803677, -2857826.01244125), ...,
     (5697075.23165051, 108150.88917568, -2857826.06827805),
     (5697033.8876135 , 108214.82367483, -2857904.71575943),
     (5697075.23165051, 108150.88917568, -2857826.06827805)]>


In [28]:
print(frame_uvw_coord)

<SkyCoord (GCRS: obstime=2457367.9577083588, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
    [(5697075.23165051, 108150.88917568, -2857826.06827042),
     (5697076.41965917, 108205.34099726, -2857822.28254363),
     (5697074.43377229, 108212.15803255, -2857826.01245487), ...,
     (5697075.23165051, 108150.88917568, -2857826.06827042),
     (5697033.88762375, 108214.82369128, -2857904.71576224),
     (5697075.23165051, 108150.88917568, -2857826.06827042)]>


In [29]:
print(new_frame_uvw_coord.cartesian - frame_uvw_coord.cartesian)
print(np.max(np.abs(new_frame_uvw_coord.cartesian.get_xyz().value - frame_uvw_coord.cartesian.get_xyz().value)))

[( 0.00000000e+00,  5.82076609e-11, -7.62986019e-06),
 (-7.03707337e-06, -1.74633169e-05,  5.58234751e-06),
 (-4.23286110e-06,  4.22501762e-06,  1.36187300e-05), ...,
 ( 0.00000000e+00,  5.82076609e-11, -7.62986019e-06),
 (-1.02445483e-05, -1.64515368e-05,  2.81725079e-06),
 ( 0.00000000e+00,  5.82076609e-11, -7.62986019e-06)] m
8.10356141300872e-05


In [30]:
frame_rel_uvw = (new_frame_uvw_coord.cartesian.get_xyz().value.T
                 - frame_telescope_location.cartesian.get_xyz().value)


In [31]:
print(frame_rel_uvw)

[[ 0.00000000e+00  5.82076609e-11 -7.62986019e-06]
 [ 1.18800162e+00  5.44518041e+01  3.78573238e+00]
 [-7.97882458e-01  6.12688611e+01  5.58291702e-02]
 ...
 [ 0.00000000e+00  5.82076609e-11 -7.62986019e-06]
 [-4.13440370e+01  6.39344992e+01 -7.86474890e+01]
 [ 0.00000000e+00  5.82076609e-11 -7.62986019e-06]]


In [32]:
print(frame_rel_uvw - uvw_rel_positions)
print(np.max(np.abs(frame_rel_uvw - uvw_rel_positions)))

[[ 0.00000000e+00  5.82076609e-11 -7.62986019e-06]
 [-7.03666612e-06 -1.74633223e-05  5.58240514e-06]
 [-4.23317920e-06  4.22501499e-06  1.36186022e-05]
 ...
 [ 0.00000000e+00  5.82076609e-11 -7.62986019e-06]
 [-1.02441180e-05 -1.64515305e-05  2.81736737e-06]
 [ 0.00000000e+00  5.82076609e-11 -7.62986019e-06]]
8.103560844574531e-05


In [33]:
phased_uvws = uvutils.phase_uvw(uvd1.phase_center_ra, uvd1.phase_center_dec, frame_rel_uvw)

In [34]:
print(phased_uvws)

[[ 5.28608899e-11 -7.25847551e-06  2.35144632e-06]
 [ 4.89526658e+01  1.09596292e+01  2.15468126e+01]
 [ 5.59749392e+01  7.73504487e+00  2.36957450e+01]
 ...
 [ 5.28608899e-11 -7.25847551e-06  2.35144632e-06]
 [ 7.53707627e+01 -7.81414894e+01  1.39833068e+01]
 [ 5.28608899e-11 -7.25847551e-06  2.35144632e-06]]


In [35]:
print(phased_uvws - uvd1.uvw_array)
print(np.max(np.abs(phased_uvws - uvd1.uvw_array)))
temp = phased_uvws - uvd1.uvw_array
wh_max = np.where(np.abs(temp) == np.max(np.abs(temp)))
print('phased baseline number with max difference:', wh_max)
print('phased baseline vector with max difference:', uvd1.uvw_array[wh_max[0], :])
print('baseline length for max difference:', np.linalg.norm(uvd1.uvw_array[wh_max[0], :]))

[[ 5.28608899e-11 -7.25847551e-06  2.35144632e-06]
 [-1.29132281e-05  1.08806604e-06 -1.47550057e-05]
 [ 5.60918021e-06  1.23160893e-05 -6.17154152e-06]
 ...
 [ 5.28608899e-11 -7.25847551e-06  2.35144632e-06]
 [-1.06515454e-05 -2.30953363e-06 -1.62709252e-05]
 [ 5.28608899e-11 -7.25847551e-06  2.35144632e-06]]
6.939918307580228e-05
phased baseline number with max difference: (array([1084]), array([0]))
phased baseline vector with max difference: [[ 681.38476562 -283.69503784  206.98391724]]
baseline length for max difference: 766.5575094644823


In [36]:
print(phased_uvws - uvd1_unphase_phase.uvw_array)
print(np.max(np.abs(phased_uvws - uvd1_unphase_phase.uvw_array)))

temp = phased_uvws - uvd1_unphase_phase.uvw_array
wh_max = np.where(np.abs(temp) == np.max(np.abs(temp)))
print('phased baseline number with max difference:', wh_max)
print('unphased-phased baseline vector with max difference:', uvd1_unphase_phase.uvw_array[wh_max[0], :])
print('baseline length for max difference:', np.linalg.norm(uvd1_unphase_phase.uvw_array[wh_max[0], :]))

[[ 1.46673237e-15 -4.31728907e-11 -1.33262476e-10]
 [-2.19342025e-05  2.80190691e-06 -4.96132592e-06]
 [-1.49770462e-05  4.82328702e-06 -1.04175784e-05]
 ...
 [ 1.46673237e-15 -4.31728907e-11 -1.33262476e-10]
 [-8.73371299e-06  5.08953363e-06 -7.61725076e-06]
 [ 1.46673237e-15 -4.31728907e-11 -1.33262476e-10]]
0.00011085121104770224
phased baseline number with max difference: (array([495]), array([0]))
unphased-phased baseline vector with max difference: [[1108.43144936   54.11025219  434.65874159]]
baseline length for max difference: 1191.8374129726237
